In [34]:
# catboost algorithms
! pip install catboost


In [35]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
# import KNN imputer and simple imputer
from sklearn.impute import SimpleImputer,KNNImputer

In [36]:
# import the data 
df=sns.load_dataset('titanic')
df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [37]:
df.isnull().sum().sort_values(ascending=False)

deck           688
age            177
embarked         2
embark_town      2
survived         0
pclass           0
sex              0
sibsp            0
parch            0
fare             0
class            0
who              0
adult_male       0
alive            0
alone            0
dtype: int64

In [38]:
# drop the deck column
df=df.drop('deck',axis=1)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [40]:
# impute the missing values in age column using KNNImputer
imputer=KNNImputer()
df['age']=imputer.fit_transform(df[['age']])

In [41]:
# impute the missing values in embarked and embark_town column using SimpleImputer
imputer=SimpleImputer(strategy='most_frequent')
df[['embarked']]=imputer.fit_transform(df[['embarked']])
df[['embark_town']]=imputer.fit_transform(df[['embark_town']])

In [42]:
# convert each category column to category
categorical_cols=df.select_dtypes(include=['object','category']).columns
df[categorical_cols]=df[categorical_cols].astype('category')


In [43]:
#split the data into X and y
X=df.drop('survived',axis=1)
y=df['survived']

In [44]:
# split the data into train and test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


In [45]:
# call the model
model=CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6)

In [46]:
# fit the model
model.fit(X_train,y_train,cat_features=categorical_cols.tolist())

0:	learn: 0.6126456	total: 347ms	remaining: 34.4s
1:	learn: 0.5508517	total: 462ms	remaining: 22.6s
2:	learn: 0.4896715	total: 566ms	remaining: 18.3s
3:	learn: 0.4369055	total: 778ms	remaining: 18.7s
4:	learn: 0.3947585	total: 978ms	remaining: 18.6s
5:	learn: 0.3547057	total: 1.16s	remaining: 18.2s
6:	learn: 0.3221894	total: 1.35s	remaining: 18s
7:	learn: 0.2911322	total: 1.56s	remaining: 17.9s
8:	learn: 0.2602813	total: 1.65s	remaining: 16.6s
9:	learn: 0.2405173	total: 1.84s	remaining: 16.6s
10:	learn: 0.2214942	total: 2.07s	remaining: 16.7s
11:	learn: 0.2011450	total: 2.23s	remaining: 16.3s
12:	learn: 0.1831262	total: 2.36s	remaining: 15.8s
13:	learn: 0.1686156	total: 2.5s	remaining: 15.4s
14:	learn: 0.1521290	total: 2.57s	remaining: 14.6s
15:	learn: 0.1413503	total: 2.73s	remaining: 14.3s
16:	learn: 0.1309976	total: 2.85s	remaining: 13.9s
17:	learn: 0.1185861	total: 2.88s	remaining: 13.1s
18:	learn: 0.1099839	total: 3.01s	remaining: 12.8s
19:	learn: 0.1022354	total: 3.2s	remaining: 

In [47]:
# predictions
y_pred=model.predict(X_test)

In [48]:
# evaluate the model
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("Accuracy Score:",accuracy_score(y_test,y_pred))

Confusion Matrix:
 [[110   0]
 [  0  69]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       110
           1       1.00      1.00      1.00        69

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179

Accuracy Score: 1.0
